# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==2.1.0
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [1]:
import os
import cases

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

## Configure plots

In [2]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    import matplotlib.pyplot as plt
    
    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)))
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)))
    plt.clf()
    

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='lightgrey', linewidth=0.25)
    return

def get_results(case_name):
    """ Get the results for the case with name `case_name`
    """
    # Make sure simulation was successful
    dslog_name = os.path.join("simulations", case_name, "dslog.txt")
    with open(dslog_name) as dslog:
        if not "Integration terminated successfully" in dslog.read():
            raise Exception("Simulation failed. Check {}".format(dslog_name))
    file_name = cases.get_result_file_name(case_name)
    return Reader(file_name, "dymola")

def get_partial_results(case_name, list_of_variables):
    """ Get a dictionary with the variable names and the time series for `list_of_variables`
    """
    reader = get_results(case_name)
    d = dict()
    read_time = True
    for v in list_of_variables:
        if read_time:
            d['time'] = reader.values(v)[0]
            read_time = False
        d[v] = reader.values(v)[1]
    return d


# ---------------------------------------------------------------------------
# helper functions and scripts

def set_cases_and_initiate_plot():
    from matplotlib.gridspec import GridSpec
    cases = ['Base case', 'Guideline 36'] # fixme
    seasons = ['Winter', 'Spring', 'Summer']
    num_cases = len(cases)
    num_seasons = len(seasons)
    
    fig = plt.figure(figsize=(6.5,8.))
    gs1 = GridSpec(80, 1)
    gs1.update(left=0.1, right=0.9, hspace=0.05)
    
    ax = list()
    ax.insert(0, fig.add_subplot(gs1[0:11,:]))
    ax.insert(1, fig.add_subplot(gs1[12:23,:]))
    ax.insert(2, fig.add_subplot(gs1[28:39,:]))
    ax.insert(3, fig.add_subplot(gs1[40:51,:]))
    ax.insert(4, fig.add_subplot(gs1[56:67,:]))
    ax.insert(5, fig.add_subplot(gs1[68:79,:]))
    
    # fig, ax = plt.subplots(nrows=num_cases*num_seasons, ncols=1, figsize = (6.5,8.))
    # fig, ax = plt.subplots(nrows=20, ncols=1, figsize = (6.5,8.))
    
    return cases, seasons, num_cases, num_seasons, fig, ax

def set_title(ax, title):
    left, width = .01, .97
    bottom, height = .01, .88
    right = left + width
    top = bottom + height
    
    title_str = r"$\it{" + title + "}$"
    ax.text(left, top,
            title_str,
            verticalalignment = 'center',
            horizontalalignment = 'left', 
            transform=ax.transAxes,
            fontsize = 6, color = 'k',
            bbox=dict(facecolor='white', alpha=0.75, edgecolor='none'))
    
    
def set_up_labels(i, ax, cases, seasons, num_cases, num_seasons, x_axis_label, y_axis_label):
    # Hide xtick labels and ticks on the upper case subplot (each basecase)
    if i % 2 == 0:
        hide_tick_labels(ax)

    # Print x axis title only below the lowest subplot
    if i  == num_cases*num_seasons - 1:
        ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))
     
    # Annotate case
    set_title(ax, cases[i % 2])
    # Annotate case
    # if i % 2 == 0:
    #     title_str = r"$\bf{" + seasons[i/2] + "}$" + ' (upper: ' + r"$\it{" + cases[i % 2] + "}$" + ', lower: ' + r"$\it{"  + cases[(i-1) % 2] + "}$" + ')'
    #     ax.set_title(title_str, # mg assign appropriate season/case
    #                  verticalalignment = 'top',
    #                  horizontalalignment = 'center', 
    #                  fontsize = 6, color = 'k')
        
    # Print legend only at the lower plot (g36 case)
    if i % 1 == 0:
        ax.legend(loc='center right', ncol=1)
    configure_axes(ax)
        
    #plt.tight_layout(h_pad=0)
    plt.tight_layout()
    #plt.subplots_adjust(hspace = .2)
        
def tem_conv_CtoF(T_in_degC):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degF = (T_in_degC)*9./5. + 32.
    
    return T_in_degF
        
def add_secondary_yaxis_for_degF(ax, time, temp_in_K):
        # Add a secondary axis with temperatures represented in F
        ax_F = ax.twinx()
        # Get limits to match with the left axis
        ax_F.set_ylim([tem_conv_CtoF(ax.get_ylim()[0]),tem_conv_CtoF(ax.get_ylim()[1])])
        # plot a "scaler" variable and make it invisible
        ax_F.plot(time, tem_conv_CtoF(temp_in_K-273.15), linewidth=0.0)
        ax_F.set_ylabel('temperature [$^\\circ$F]')
        configure_axes(ax_F)
        #ax.grid(False)
        #ax.xaxis.grid()
        
def hide_tick_labels(ax):
    '''Removes labels and ticks. Kwargs: bottom controls the ticks, labelbottom the tick labels
    '''
    ax.tick_params(axis = 'x',labelbottom='off',bottom='off')

## Read annual results

This section also clears the old results to free up memory.

In [3]:
# Free up storage, then read new data
import gc
#r_all_without_diverse_loads = list()
#r_all_with_diverse_loads = list()
#r_all = list()
gc.collect()

print("Cleaned up memory.")

Cleaned up memory.


In [4]:
list_of_variables = ['ETot.y',
                     "CPUtime"]
#r_base=get_partial_results('base', list_of_variables)
r_base=get_results('base')
print("Read base results")


Read base results


In [5]:
print(f"Total energy use: {r_base.max('ETot.y')/3600/1E9:.2f} GWh")


Total energy use: 0.05 GWh


### Energy use


In [6]:
def plot_energy(results : list):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    import numpy as np
    
    plt.clf()    

    # Conversion to kWh
    conv = 1/3600.
    width = 0.5       # the width of the bars: can also be len(x) sequence

    EPumDis    = [0., 0.]
    EPumETS = [0., 0.]
    EPumDryCoo = [0., 0.]
    
    idx = [0, 1]
    for i in idx:
        res = results[i]
        # fixme: add the right variables
        EPumDis[i]    = res.max('EPumDis.y') * conv
        EPumETS[i]    = res.max('EPumETS.y') * conv
        EPumDryCoo[i] = res.max('EPumDryCoo.y') * conv

    EPumDryCooBas = np.add(EPumDis, EPumETS)
        
    p1 = plt.bar(idx, EPumDis, width, color='r')
    p2 = plt.bar(idx, EPumETS, width, bottom=EPumDis, color='g')
    p3 = plt.bar(idx, EPumDryCoo, width, bottom=EPumDryCooBas, color='b')
        
    plt.ylabel('site electricity use [kWh/(m2  a)] FIXME')
    plt.xticks([0, 1], ('base case', 'base case'))
    plt.tick_params(axis=u'x', which=u'both',length=0)

    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0]), \
               ('pump district', 'pump ETS', 'pump dry cooler'), \
              loc='upper right')
    
    save_plot(plt, "energy")
    
    # Write result to console and file
##    eSit = [0, 0]
##    for i in [0, 1]:
##        eSit[i] = hea[i]+cooSen[i]+cooLat[i]+fan[i]

##    str = """\
##.. table:: Heating, cooling, fan and total site HVAC energy, and savings of guideline 36 case versus base case.
##
##   ===================================== ===================================== ====================================== ====================================== =====
##   :math:`E_{{h}} \quad [kWh/(m^2\,a)]`    :math:`E_{{c}} \quad [kWh/(m^2\,a)]`    :math:`E_{{f}} \quad [kWh/(m^2\,a)]`     :math:`E_{{tot}} \quad [kWh/(m^2\,a)]`     [%]
##   ===================================== ===================================== ====================================== ====================================== =====
##   {:37.4} {:37.4}  {:37.4}  {:37.4}   
##   {:37.4} {:37.4}  {:37.4}  {:37.4}  {:4.3} 
##   ===================================== ===================================== ====================================== ====================================== =====
##  
##  """.format(\
##            hea[0], coo[0], fan[0], eSit[0], \
##            hea[1], coo[1], fan[1], eSit[1], (1-eSit[1]/eSit[0])*100.)
    def save_rst(str, file_name):
        ''' Save the string `str` to the rst file `file_name.rst`
        '''
        print(str)
        with open(os.path.join("img", "{}.rst".format(file_name)), "w") as fil:
            fil.write(str)
        
##    save_rst(str, "site_energy")
plot_energy([r_base, r_base])

### Diagnostic output

In [7]:
print(f"CPUtime, base {r_base.max('CPUtime')/3600.:.2f} h")

CPUtime, base 0.05 h


In [8]:
plt.clf()

roo_lis = [{"var": 'nor', "name": 'north'},
           {"var": 'wes', "name": 'west'},
           {"var": 'sou', "name": 'south'},
           {"var": 'eas', "name": 'east'},
           {"var": 'cor', "name": 'corridor'}]
nRoo = len(roo_lis)
iRoo = 1
iPlt = 1
for roo in roo_lis:
    iSim = 1
    for sim in [{"res": r_base_annual_without_diverse_loads, 'name': 'base case'}, 
                {"res": r_g36_annual_without_diverse_loads, 'name': 'guideline36'}]:
        plt.subplot(nRoo, 2, iPlt)
        plt=Plotter.boxplot(t=sim['res']['time'], \
                    y=sim['res']['flo.{}.air.vol.T'.format(roo['var'])]-273.15, \
                    increment=3600, nIncrement=24)
        #plt.set_facecolor('mistyrose')    
        #plt.patch.fill_between(t, TSetHea-273.15, y2=TSetCoo-273.15, color='white')
        # Decorate, save and show the plot
        if iRoo is nRoo:
            plt.xlabel('Time [h]') 
        if iSim is 1:
            plt.ylabel(u'$T_{room}$ [$^\circ$C]')
        plt.grid()
        plt.xlim(6.5,19.5)
        plt.ylim(18, 26)
        plt.yticks([18, 20, 22, 24, 26])

        plt.title('{} zone, {}'.format(roo['name'], sim['name']))
        iSim+=1
        iPlt+=1
    iRoo+=1
plt.tight_layout()
save_plot(plt, "roomTemperatures_boxplot")
plt.show()

<>:25: SyntaxWarning: invalid escape sequence '\c'
<>:24: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
<>:25: SyntaxWarning: invalid escape sequence '\c'
<>:24: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
/var/folders/v2/6wff_z7125dbkwqqtb7hw_fw0000gn/T/ipykernel_30152/3219937733.py:25: SyntaxWarning: invalid escape sequence '\c'
  plt.ylabel(u'$T_{room}$ [$^\circ$C]')
/var/folders/v2/6wff_z7125dbkwqqtb7hw_fw0000gn/T/ipykernel_30152/3219937733.py:24: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  if iSim is 1:
/var/folders/v2/6wff_z7125dbkwqqtb7hw_fw0000gn/T/ipykernel_30152/3219937733.py:25: SyntaxWarning: invalid escape sequence '\c'
  plt.ylabel(u'$T_{room}$ [$^\circ$C]')


NameError: name 'r_base_annual_without_diverse_loads' is not defined